In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q git+https://github.com/openai/CLIP.git transformers bitsandbytes

     |████████████████████████████████| 5.8 MB 33.7 MB/s 
     |████████████████████████████████| 62.5 MB 1.2 MB/s 
     |████████████████████████████████| 53 kB 1.9 MB/s 
     |████████████████████████████████| 7.6 MB 50.0 MB/s 
     |████████████████████████████████| 182 kB 77.8 MB/s 


In [ ]:
!cp -r /content/drive/MyDrive/Olimpiads/nto_hack_2022/V2/data /content/
!unzip -q /content/drive/MyDrive/Olimpiads/nto_hack_2022/V2/videos_train.zip 
!mv videos_train /content/data

!cp -i /content/drive/MyDrive/Olimpiads/nto_hack_2022/V2/config.py /content/
!cp -i /content/drive/MyDrive/Olimpiads/nto_hack_2022/V2/utils.py /content/
!cp -i /content/drive/MyDrive/Olimpiads/nto_hack_2022/V2/model.py /content/

In [ ]:
import bitsandbytes as bnb
import gc
import io
import os
import random
import numpy as np
from tqdm import tqdm
import pandas as pd
import cv2
import sys

import torchvision
import transformers
import torch
from torch.nn import functional as nnf
from torch.utils.data import DataLoader
from transformers import AdamW, get_linear_schedule_with_warmup
from torch.cuda.amp import autocast

from utils import *
from model import *
from config import CFG

from transformers.optimization import Adafactor, AdafactorSchedule
from torch.utils.checkpoint import checkpoint_sequential
import warnings
warnings.simplefilter('ignore')

import time

In [ ]:
CFG.batch_size

24

In [ ]:
def get_caption(prefix, model, device, tokenizer, prompt=''):
    prefix = prefix.to(device)
    with torch.no_grad():

        prefix_embed = model.clip_project(prefix).reshape(len(prefix), CFG.prefix_length, -1)

        answers = []

        for x in range(len(prefix_embed)):

            start = time.time()

            cur_prefix_embed = prefix_embed[x].unsqueeze(0).to('cpu')
            
            print('first', time.time()-start)
            
            if prompt:
                generated_text_prefix = generate2(model, tokenizer, prompt=prompt, embed=cur_prefix_embed)
            else:
                generated_text_prefix = generate2(model, tokenizer, embed=cur_prefix_embed)
            
            print('second', time.time()-start)
        
            answers.append(generated_text_prefix.replace('\n',' ').replace('<|endoftext|',''))

    return [x[len(prompt):].strip() for x in answers]

# def get_ans(model, clip_emb, prompt, device, tokenizer):
#     output = get_caption(clip_emb, model, device, tokenizer, prompt=prompt)
#     return output

In [ ]:
def train(train_loader, model, optimizer, scheduler, device, epoch):
    loss_avg = AverageMeter()
    
    model = model.to(device)
    model.train()

    progress = tqdm(total=len(train_loader))
    for idx, (tokens, mask, prefix) in enumerate(train_loader):
        model.zero_grad()
        tokens, mask, prefix = tokens.to(device), mask.to(device), prefix.to(device, dtype=torch.float32)
        
        outputs = model(tokens, prefix, mask)
        logits = outputs.logits[:, CFG.prefix_length-1: -1]

        loss = nnf.cross_entropy(logits.reshape(-1, logits.shape[-1]), tokens.flatten(), ignore_index=0)

        segments = 2

        # out = checkpoint_sequential(modules, segments, input_var)

        loss.backward()    
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        
        clipping_value = 0.5 # arbitrary value of your choosing
        #torch.nn.utils.clip_grad_norm(model.parameters(), clipping_value)

        loss_avg.update(loss.item(), len(mask))
        progress.set_description(f"loss: {loss_avg.avg:.5f}")
        progress.update()
        

        # del tokens
        # del mask
        # del prefix
        torch.clear_autocast_cache()
        torch.cuda.empty_cache()
    progress.close()

    return model


def valid(model, valid_loader, device, gt):
    loss_avg = AverageMeter()
    model.eval()
    progress = tqdm(enumerate(valid_loader), total=len(valid_loader))
    tokenizer = GPT2Tokenizer.from_pretrained(CFG.backbone)
    

    all_answers = []
    for idx, (tokens, mask, prefix) in progress:
        tokens, mask, prefix = tokens.to(device), mask.to(device), prefix.to(device, dtype=torch.float32)
        answer = get_caption(prefix, model, device, tokenizer, prompt='Caption: ')
        all_answers.append(answer)
    score = bleu_metric(gt, np.concatenate(all_answers))
    return score
    

import nltk
def bleu_metric(ground_truth, prediction):
    scores = []
    for gt, pred in zip(ground_truth, prediction):
        if type(pred)==str and type(gt)==str:
            score = nltk.translate.bleu_score.sentence_bleu([gt.lower().split()], pred.lower().replace('<|endoftext|>','').split(), weights = (0.5, 0.5))
        scores+=[score]
    return np.array(scores).mean()*100

In [ ]:
def main():
    valid_df = pd.read_csv(CFG.valid_df_path)
    train_ds = ClipCocoDataset(CFG.train_features_path, CFG.prefix_length)
    valid_ds = ClipCocoDataset(CFG.valid_features_path, CFG.prefix_length)

    train_loader = DataLoader(train_ds, batch_size=CFG.batch_size, shuffle=True, drop_last=True)
    valid_loader = DataLoader(train_ds, batch_size=CFG.batch_size, shuffle=False)

    model = ClipCaptionModel(prefix_length = CFG.prefix_length, backbone = CFG.backbone)
    device = torch.device('cuda') # xm.xla_device()

    if not os.path.exists(CFG.out_dir):
        os.makedirs(CFG.out_dir)

    # model.load_state_dict(torch.load('/content/drive/MyDrive/Olimpiads/nto_hack_2022/V2/coco_flickr-pretrained.pt', map_location='cpu')) 

    model.load_state_dict(torch.load('/content/drive/MyDrive/Olimpiads/nto_hack_2022/V2/v2_1.pt', map_location='cpu'))
    model = model.to(device)
   
    #model = freeze(model)

    model.train()
    optimizer = AdamW(model.parameters(),lr=CFG.learning_rate, betas=(0.9, 0.995))
    #optimizer = bnb.optim.Adam8bit(model.parameters(), lr=0.001, betas=(0.9, 0.995))
    #optimizer = SM3(model.parameters(),lr=args.lr)
    #Adafactor(model.parameters(),scale_parameter=True, relative_step=True, warmup_init=True, lr=None)

    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=CFG.warmup_steps, num_training_steps=CFG.epochs * len(train_loader))
    #AdafactorSchedule(optimizer)#num_training_steps=epochs * len(train_loader)

    for epoch in range(1, 1+CFG.epochs):
        train(train_loader, model, optimizer, scheduler, device, epoch)
        valid(model, valid_loader, device, valid_df.caption.tolist()) 

        if epoch % CFG.save_every==0:
            torch.save(model.state_dict(),os.path.join(CFG.out_dir, f"{CFG.model_name}.pt"))
    
main()

### Validation

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as nnf
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from tqdm import tqdm, trange
import os
import pickle
import sys
import argparse
import json
from typing import Tuple, Optional, Union
#from torch.cuda.amp import autocast
import io
import os
import PIL
import random
import numpy as np
import torch
import torchvision
import transformers
import more_itertools
import numpy as np
import matplotlib.pyplot as plt
#from tqdm import tqdm
import pandas as pd
from torch.utils.data import Dataset
#from tqdm import tqdm
from dataclasses import dataclass, field
import torchvision.transforms as T
import torchvision.transforms.functional as TF
import cv2
from PIL import Image
import clip

import transformers

from utils import *
from model import *
import re

import warnings
warnings.simplefilter('ignore')

import time

In [ ]:
def get_caption(prefix, prompt=''):
        prefix = prefix.to(device)
        display(prefix.shape)
        with torch.no_grad():

            prefix_embed = model.clip_project(prefix).reshape(1, prefix_length, -1)

            start = time.time()

            if prompt:
                generated_text_prefix = generate2(model, tokenizer, prompt=prompt, embed=prefix_embed)
            else:
                generated_text_prefix = generate2(model, tokenizer, embed=prefix_embed)

            print(time.time() - start)

        return generated_text_prefix.replace('\n',' ').replace('<|endoftext|','')

def get_ans(clip_emb, prompt):
        output = get_caption(clip_emb, prompt=prompt)
        ans = output[len(prompt):].strip()
        return {'answer': ans}


#from tqdm import tqdm, trange



# parser = argparse.ArgumentParser()
# parser.add_argument('--input_path', default='./input_test.csv', type=str, help='input path')
# parser.add_argument('--video_path', default='./videos_val/', type=str, help='input path')
# parser.add_argument('--output_path', default='./output/', type=str, help='config path')
# args = parser.parse_args()

config = dict(
        model_path = '/content/drive/MyDrive/Olimpiads/nto_hack_2022/V2/v2_1.pt',
        video_path = '/content/data/videos_train/',
        val_path = '/content/data/new_valid.csv',
        gpt = 'gpt2',
        prefix_len = 35
    )


prefix_length = config['prefix_len']#40

device = 'cuda'
clip_model, preprocess = clip.load("ViT-L/14@336px", device=device, jit=False)
clip_model.to(device)

tokenizer = GPT2Tokenizer.from_pretrained(config['gpt'])


model_path = config['gpt']
model = ClipCaptionModel(prefix_length = prefix_length, backbone = config['gpt'])

model.load_state_dict(torch.load(config['model_path'], map_location='cpu'))
model.to(device)

out_path = 'Features_val.pkl'


val_embeddings = []
val_captions = []

input_test = pd.read_csv(config['val_path'])

c = 0
for p in tqdm(input_test.paths):
         #print(p)
         #n= df_eval.iloc[i, 0]#, df_eval.iloc[i, 1]
    text = f'Caption:'
    path = f'{config["video_path"]}{p}'
    try:
        video = read_video(path, transform=None,frames_num=1)

        i = image_grid(video,1,1)
        image = preprocess(i).unsqueeze(0).to(device)

        # image = make_images(video)
        # new_images = []
        # for img in image:
        #     new_images.append(preprocess(img).unsqueeze(0))

        with torch.no_grad():
            prefix = clip_model.encode_image(image).to(device, dtype=torch.float32)
            # prefix_embed = model.clip_project(prefix).reshape(1, prefix_length, -1)
        val_embeddings.append(prefix)
        val_captions.append(text)
    except Exception as e:
        print(e)

    c+=1
    if c > 50:
        break

answers = []
for i in tqdm(range(len(val_embeddings))):
        emb = val_embeddings[i]

        #qid = df_eval.iloc[i, 2]
        ans = get_ans(emb, 'Caption: ')
        answers.append(ans['answer'])


df = pd.DataFrame({'captions':answers})
df.to_csv('/content/data/answer.csv', index=False)

  0%|          | 0/51 [00:00<?, ?it/s]

torch.Size([1, 768])

  2%|▏         | 1/51 [00:00<00:15,  3.23it/s]

0.3034696578979492


torch.Size([1, 768])

...

 98%|█████████▊| 50/51 [00:12<00:00,  5.68it/s]

0.11292529106140137


torch.Size([1, 768])

100%|██████████| 51/51 [00:12<00:00,  3.98it/s]

0.08947086334228516


In [ ]:
path_pred = '/content/data/answer.csv'
df_pred = pd.read_csv(path_pred)
new_preds = []
for x in df_pred['captions']:
    a = ''
    if x is not np.nan:
        for y in x:
            if re.findall(r'[A-Za-z0-9 \.\,\-]', y):
                a += y
    new_preds.append(a)
df_pred['captions'] = new_preds
df_pred.to_csv('/content/data/new_answer.csv', index=False)

In [ ]:
import pandas as pd
import nltk
import numpy as np

path_gt = '/content/data/new_valid.csv'
path_pred = '/content/data/answer.csv'
df_eval = pd.read_csv(path_gt)
df_pred = pd.read_csv(path_pred)
scores = []
for pred, gt in zip(df_eval.caption, df_pred.captions):
    if type(pred)==str and type(gt)==str:
        score = nltk.translate.bleu_score.sentence_bleu([gt.lower().split()], pred.lower().replace('<|endoftext|>','').split(), weights = (0.5, 0.5))
    scores+=[score]
ans = np.array(scores).mean()*100
print(round(ans, 5))

7.11994


In [ ]:
for i in range(50):
    print(df_eval.caption.tolist()[i])
    print(df_pred.captions.tolist()[i])
    print()

Beautiful blue and yellow flowers in an arrangement close-up, pan shot.
Bouquet of little pink flowers of different colors and sizes

Dirty car bumper
Cars driving in a highway

Girl running on the street with masks, front view and close to the shot, with the panorama of the street out of focus in the background, with cars, trees, the sidewalk and the sun lighting up.
A woman in a black mask runs through the park with her hands on her feet, surrounded by trees and buildings.

Woman with a hat watching the ocean
A calm sea and mountains

Married couple receiving at the door of their house another neighbor couple who arrived with welcome gifts, while smiling happily.
Happy businessman gives a thumbs up as he hugs his girlfriend during a break on a date.

View under a bridge that crosses a river in a big city, where some people go sailing by boat underneath.
A group of people rowing on a river surrounded by trees and buildings, during a sunny day.

Guangzhou illuminated cityscape with clo